In [330]:
import docx
import numpy as np
from docx.enum.section import WD_SECTION, WD_ORIENT
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import RGBColor, Pt
from docx.enum.table import WD_TABLE_DIRECTION, WD_TABLE_ALIGNMENT, WD_CELL_VERTICAL_ALIGNMENT
from math import floor
from datetime import date
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml

import docx2pdf


In [331]:
from docx.enum.style import WD_STYLE_TYPE
styles = docx.Document().styles
paragraph_styles = [ s for s in styles if s.type == WD_STYLE_TYPE.CHARACTER]
for style in styles:
    print(style.name)

Normal
Header
Header Char
Footer
Footer Char
Heading 1
Heading 2
Heading 3
Heading 4
Heading 5
Heading 6
Heading 7
Heading 8
Heading 9
Default Paragraph Font
Normal Table
No List
No Spacing
Heading 1 Char
Heading 2 Char
Heading 3 Char
Title
Title Char
Subtitle
Subtitle Char
List Paragraph
Body Text
Body Text Char
Body Text 2
Body Text 2 Char
Body Text 3
Body Text 3 Char
List
List 2
List 3
List Bullet
List Bullet 2
List Bullet 3
List Number
List Number 2
List Number 3
List Continue
List Continue 2
List Continue 3
macro
Macro Text Char
Quote
Quote Char
Heading 4 Char
Heading 5 Char
Heading 6 Char
Heading 7 Char
Heading 8 Char
Heading 9 Char
Caption
Strong
Emphasis
Intense Quote
Intense Quote Char
Subtle Emphasis
Intense Emphasis
Subtle Reference
Intense Reference
Book Title
TOC Heading
Table Grid
Light Shading
Light Shading Accent 1
Light Shading Accent 2
Light Shading Accent 3
Light Shading Accent 4
Light Shading Accent 5
Light Shading Accent 6
Light List
Light List Accent 1
Light List 

## playing with the header

In [332]:
def Generate_A4(logo_path, centered_text, right_hand_text):
    testDoc = docx.Document()
    # new_section = testDoc.add_section(WD_SECTION.CONTINUOUS)
    section = testDoc.sections[0]

    section.orientation = WD_ORIENT.PORTRAIT
    section.page_width = Inches(8.3)
    section.page_height = Inches(11.7)

    section.left_margin = Inches(0.5)
    section.right_margin = Inches(0.5)
    section.top_margin = Inches(0.5)
    section.bot_margin = Inches(0.5)

    docHeader = section.header
    headerParagraph = docHeader.paragraphs[0]
    headerParagraph.text = ""
    headerParagraph.style = testDoc.styles["Header"]

    newRun = headerParagraph.add_run(style=testDoc.styles["Heading 1 Char"])
    newRun.add_picture (logo_path, width=Inches(1.5))
    newRun.add_tab()
    newRun.add_text("بسم الله الرحمن الرحيم")
    newRun.add_tab()
    newRun.add_text('إدارة المتابعة')

    return testDoc
# headerParagraph.



# to add a picture:
#testDoc.add_picture('./data/logo.png', width=Inches(1.5))


testDoc = Generate_A4(logo_path='./data/logo.png', centered_text='', right_hand_text='')
testDoc.save('testDoc.docx')



In [333]:
def addTitle(document, text, color):
    center_paragraph = document.add_paragraph(text)
    paragraph_format = center_paragraph.paragraph_format
    paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

    center_paragraph.style = document.styles['Heading 1']
    center_paragraph.style.font.size = Pt(26)
    center_paragraph.style.font.underline = True
    center_paragraph.style.font.color.rgb = color



addTitle(testDoc, text=f"""بيان  بالتمام عن اليوم الأحد بتاريخ {date.today()}""", color=RGBColor(0x00, 0x00, 0x00))



In [334]:
def addArabicText(document, text, underline:bool, size, color_RGB):
    new_par = testDoc.add_paragraph(text)
    paragraph_format = new_par.paragraph_format
    paragraph_format.alignment = WD_ALIGN_PARAGRAPH.RIGHT
    new_par.style = testDoc.styles['Heading 1']
    new_par.style.font.size = size
    new_par.style.font.underline = underline
    new_par.style.font.color.rgb = RGBColor(*color_RGB)

addArabicText(testDoc, text="أولاً السادة الضباط:", underline=True, size=Pt(26), color_RGB = (0x00, 0x00, 0x00))

In [335]:
def insertCharInsideString(sourceString, addedChar, index):
    return sourceString[:index+1] + addedChar + sourceString[index+1:]

def addDummyChars(text:str, text_size, bounding_inches):
    unlinkable_chars = list('ةاذدور زأإءآىؤى')
    middle_index = floor(len(text) / 2)
    distance_from_middle = 0
    while(text[middle_index + distance_from_middle] in unlinkable_chars and text[middle_index - distance_from_middle] in unlinkable_chars):
        distance_from_middle += 1
    

    non_preceded_chars = list(' ء')
    additional_distance = 1
    print(text)
    while(text[middle_index + distance_from_middle + additional_distance] in non_preceded_chars and text[middle_index - distance_from_middle - additional_distance] in non_preceded_chars):
            print(text[middle_index + distance_from_middle + additional_distance])
            additional_distance += 1
            if(text[middle_index + distance_from_middle + additional_distance] == 'ل' and text[middle_index + distance_from_middle + additional_distance+1] in list("أ ا إ آ")):
                 additional_distance += 1
    
    additional_distance = 0 if additional_distance == 1 else additional_distance

    targetIndex = (middle_index + distance_from_middle + additional_distance) if text[middle_index + distance_from_middle + additional_distance] not in unlinkable_chars else (middle_index - distance_from_middle - additional_distance)


    while(Pt(text_size)*len(text) < Inches(bounding_inches+5)):
        text = insertCharInsideString(text, 'ـ', targetIndex)

    return text






# Testing this function on certain names and words
testTxt = 'الموجود'
print(addDummyChars(testTxt, 12, 4))

testTxt = 'أيمن محمد رضا محمد'
print(addDummyChars(testTxt, 12, 8))

testTxt = 'أدهم محمد الحلي'
print(addDummyChars(testTxt, 12, 8))

testTxt = 'خالد محمد هاشم'
print(addDummyChars(testTxt, 12, 8))

testTxt = 'محمود صلاح اسماعيل'
print(addDummyChars(testTxt, 12, 8))
print('الموجــــود')

الموجود
الموجـــــــــــــــــــــــــــــــــــــــــــــــود
أيمن محمد رضا محمد
أيمن محمد رضــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــا محمد
أدهم محمد الحلي
أدهم محمـــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــد الحلي
خالد محمد هاشم
خالد محمــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــد هاشم
محمود صلاح اسماعيل
محمود صلاحــــــــــــــــــــــــــــــــــــــــــــــــــــــــــــ اسماعيل
الموجــــود


In [336]:
def set_table_header_bg_color(cell, color):
    """
    set background shading for Header Rows
    """
    tblCell = cell._tc
    tblCellProperties = tblCell.get_or_add_tcPr()
    constructed_Hex_color = ''
    for c in color:
        hex_str = str(hex(c))[2:]
        if(len(hex_str) == 1):
            hex_str = '0' + hex_str
        constructed_Hex_color = constructed_Hex_color + hex_str

    xml_string = r'<w:shd {} w:fill="'+constructed_Hex_color+'"/>'
    
    shading_elm = parse_xml(xml_string.format(nsdecls('w')))
    tblCellProperties.append(shading_elm)
    return cell




def addTableFormatted(document, num_rows, num_cols, headers, input_rows, fill_entire_width:bool, header_color_rgb= None, header_font_size=24, cells_font_size = 20, font_color = (0, 0, 0)):
    #some assertions
    for c in header_color_rgb:
        if c > 255 or c < 0:
            raise TypeError
    for c in font_color:
        if c > 255 or c < 0:
            raise TypeError
    
    
    table = document.add_table(rows=num_rows, cols=num_cols)
    table.autofit=True
    table.style = 'Table Grid'
    table.table_direction=WD_TABLE_DIRECTION.RTL
    table.allignment = WD_TABLE_ALIGNMENT.RIGHT


    #decorating the header row and filling it
    for row in table.rows:
        row.height = Pt(header_font_size)

    while (len(headers) > num_cols):
        table.add_column()

    for i, header in enumerate(headers):
        par = table.cell(0, i).add_paragraph(header)
        paragraph_format = par.paragraph_format
        paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        par.style.font.size = Pt(header_font_size)
        par.style.font.bold = True
        par.style.font.underline = False
        table.cell(0,i)._element = set_table_header_bg_color(table.cell(0,i), header_color_rgb)
        par.style.font.color.rgb = RGBColor(*font_color) if font_color else RGBColor(0,0,0)

        #resizing the cell to combat large text in smaller cells
        if(par.style.font.size.inches * len(header) > table.cell(0, i).width):
            table.cell(0, i).width = Inches(par.style.font.size.inches * len(header))

        #modifying the text to combat small text in larger cells, by adding dummy "ـ"
        par.text = addDummyChars(header, par.style.font.size.pt, table.cell(0, i).width.inches)

    for i, input_row in enumerate(input_rows):
        for j, newcell in enumerate(table.row_cells(i+1)): #i+1 so as to skip the first row, aka the header row
                # newcell.text = input_rows[i][j]
                par = newcell.add_paragraph(input_rows[i][j])
                newcell.vertical_alignment = WD_CELL_VERTICAL_ALIGNMENT.CENTER
                par.style.font.size = Pt(cells_font_size)
            

        


In [337]:

addTableFormatted(testDoc, 4, 3, ['القوة', 'الموجود', 'الغياب'], [("3", "2", "1"), ("3", "2", "1"), ("3", "2", "1")], fill_entire_width=True, header_color_rgb=(0xd9, 0xd9, 0xd9), header_font_size=25, cells_font_size=12, font_color=(0, 0, 0))



القوة
الموجود
الغياب


In [338]:
print(docx.Document().add_table(3,3).cell(0,0).width.inches)

print(docx.Document().styles['Heading 1'].font.size.inches)

2.0
0.19444444444444445


In [339]:
print(testDoc.styles['Title'].font.__dir__())

['_element', '_parent', '_r', '__module__', '__doc__', '__init__', 'all_caps', 'bold', 'color', 'complex_script', 'cs_bold', 'cs_italic', 'double_strike', 'emboss', 'hidden', 'highlight_color', 'italic', 'imprint', 'math', 'name', 'no_proof', 'outline', 'rtl', 'shadow', 'size', 'small_caps', 'snap_to_grid', 'spec_vanish', 'strike', 'subscript', 'superscript', 'underline', 'web_hidden', '_get_bool_prop', '_set_bool_prop', '__eq__', '__ne__', 'element', 'part', '__dict__', '__weakref__', '__hash__', '__new__', '__repr__', '__str__', '__getattribute__', '__setattr__', '__delattr__', '__lt__', '__le__', '__gt__', '__ge__', '__reduce_ex__', '__reduce__', '__getstate__', '__subclasshook__', '__init_subclass__', '__format__', '__sizeof__', '__dir__', '__class__']


## saving the docx and hence converting the docx to a corresponding PDF file

In [340]:
testDoc.save('testDoc.docx')
docx2pdf.convert('testDoc.docx', "output.pdf")

100%|██████████| 1/1 [00:03<00:00,  3.29s/it]
